In [1]:
from __future__ import annotations

import attrs

from laboneq.serializers.base import VersionedClassSerializer
from laboneq.simple import QuantumElement
from laboneq.serializers.serializer_registry import serializer
from laboneq.serializers.types import (
    DeserializationOptions,
    JsonSerializableType,
    SerializationOptions,
)
from laboneq.serializers.core import import_cls

# New serializers in LabOneQ

The new LabOne Q serializer allows for more flexibility in the serialization process.

It is now possible to serialize and deserialize objects that are not part of the standard library. Such features are useful, for example, when users want to implement new [quantum elements](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/03_sections_pulses_and_quantum_operations/tutorials/04_quantum_elements.html) or [quantum operations](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/03_sections_pulses_and_quantum_operations/concepts/08_quantum_operations.html) classes.

When the API of the objects changes, new versions of the serialization can be added to the serializer. The serializer can then handle the different versions of the objects automatically and hence maintain the backward compatibility.

The new serializer engine provides generic methods for saving and loading any objects that are supported for serialization. Let's import them.


In [2]:
from laboneq.serializers import from_dict, to_dict, to_json, from_json, save, load

If you want to serialize an object to a dictionary form, just call `to_dict()`. For example, to serialize a `QuantumElement` object ,

In [3]:
q0 = QuantumElement("q0")
serialized_q0 = to_dict(q0)
serialized_q0

{'__serializer__': 'laboneq.serializers.implementations.QuantumElementSerializer',
 '__version__': 1,
 '__data__': {'quantum_element_class': 'laboneq.dsl.quantum.quantum_element.QuantumElement',
  'uid': 'q0',
  'signals': {},
  'parameter_class': 'laboneq.dsl.quantum.quantum_element.QuantumParameters',
  'parameters': {}},
 '__creator__': ['laboneq', '2.49.0']}

It worths mentioning what is contained in the returned dictionary. 
Perhaps the most important field is `__data__` which contains information required to initialize the serialized objects again.
The meta field `__serializer__`, `__version__` helps to reload the objects with correct versioning. We will learn more about these fields in the next section.
`__creator__` tells us the version of LabOne Q that performs the serialization. This is not crucial for the serialization process but could be useful for troubleshooting. 

Please note that the returned dictionary is not directly Json-serializable as it may contain numpy arrays which requires a third party library such as `orjson` to convert it to a json. If users would like to serialize the object to json directly, consider using `to_json` that will be explained shortly.

In [ ]:
serialized_q0

And to load the object back,

In [ ]:
loaded_q0 = from_dict(serialized_q0)
loaded_q0

`to_json` and `from_json` can be used in a similar way to convert objects to/from byte strings. Serializing objects to a byte strings could be useful when we want to send them over a network.

In [ ]:
byte_string_q0 = to_json(q0)
byte_string_q0

In [ ]:
loaded_q0 = from_json(byte_string_q0)
loaded_q0

Last but not least, we can convert objects to byte strings and save it to a file by using `save`.

In [8]:
save(q0, "q0.json")

And to load it back,

In [ ]:
loaded_q0 = load("q0.json")
loaded_q0

Another different aspect of the new serializer is that the serializing is decoupled from data classes. A serializer class must be written for each data class that we want to support serialization for.

LabOne Q provides a global default serializer registry that already contains serializers for some of the LabOne Q objects.

The currently supported objects for serialization are:
* Python built-in data types
* Numpy arrays
* QPU
* QuantumParameters
* QuantumElement
* RunExperimentResults
* Workflow
* Workflow Namespace

Note that when encountering LabOne Q objects that are not in this list, and hence not supported directly by one of the new serializers, the new serialization engine will resort to use the classic versions of the serializers to do the job. In the future versions of LabOne Q, we are going to replace the old serializers with the new ones, so we won't cover the old serializers more in this tutorial.

In the following sections, we will learn how to write a new serializer and add it into the serializer registry.

# How to write and register new serializers

A serializer must be written for any new class that does not have an existing serializer implemented for itself or its parent classes.

We will learn how to write a new serializer class by actually writing one for the `QuantumElement` class and call it `QuantumElementSerializer`.

<div class="alert alert-block alert-info">
<b>Note:</b>
The serializer for the `QuantumElement` class is already implemented in the `laboneq` package. You can immediately save and load `QuantumElement` without writing a new one.
This is just an example to illustrate how to write a new serializer.
</div>





The new serializer class must inherit from `VersionedClassSerializer` and must define the two class variables `SERIALIZER_ID` and `VERSION`.

Specifying `SERIALIZER_ID` as the path for the class could be helpful when the serializer is not registered in the global `serializer_registry`. In this case, the serialization engine imports the class of the object using the path specified in `SERIALIZER_ID`.

We should not forget to add our new serializer to `serializer_registry`. This can be done via the decorator `@serializer`. 

In [10]:
@serializer(types=[QuantumElement], public=True)
class QuantumElementSerializer(VersionedClassSerializer[QuantumElement]):
    SERIALIZER_ID = "laboneq.serializers.implementations.QuantumElementSerializer"
    VERSION = 1

In addition, we need to implement the following methods for the serializer: `to_dict` and `from_dict_vx`, where `x` is the version of the serializer.

Let's first look at the `to_dict` method, which is supposed to return a dictionary with three compulsory fields: `__serializer__`, `__version__`, and `__data__`. 

The former two are metadata and required for selecting the right serializer with the correct version.

On the other hand, `__data__` contains information required for loading the objects properly. Peeking at the definition of the  `QuantumElement` class, we know that we need the following attributes for creating a `QuantumElement` instance: `uid`, `signals`, and `parameters`. 

Because both `uid` and `signals` are Python primitive data types, we could simply assign the corresponding values `obj.uid` and `obj.signals`. 

We, however, need both the class name and the serialized form for abstract data types such as `parameters`. 

Furthermore, we should not forget about the name of the class we are serializing which goes into `quantum_element_class`.


In [11]:
@classmethod
def to_dict(
    cls, obj: QuantumElement, options: SerializationOptions | None = None
) -> JsonSerializableType:
    return {
        "__serializer__": cls.serializer_id(),
        "__version__": cls.version(),
        "__data__": {
            "quantum_element_class": f"{obj.__class__.__module__}.{obj.__class__.__name__}",
            "uid": obj.uid,
            "signals": obj.signals,
            "parameter_class": f"{obj.parameters.__class__.__module__}.{obj.parameters.__class__.__name__}",
            "parameters": attrs.asdict(obj.parameters),
        },
    }

Let's continue with the deserializing method `from_dict_v1`, which initializes a new `QuantumElement` object with inputs taken from the fields of `__data__`

In [12]:
@classmethod
def from_dict_v1(
    cls,
    serialized_data: JsonSerializableType,
    options: DeserializationOptions | None = None,
) -> QuantumElement:
    data = serialized_data["__data__"]
    qe_cls = import_cls(data["quantum_element_class"])
    param_cls = import_cls(data["parameter_class"])
    return qe_cls(
        uid=data["uid"],
        signals=data["signals"],
        parameters=param_cls(**from_dict(data["parameters"])),
    )

## How to add a new version to an existing serializer and how to deal with API changes.

Now let's imagine we'd like to rename `parameters` to `attributes`. This certainly breaks the backwards compatibility of `QuantumElement` class and requires us to update its serializer, `QuantumElementSerializer`.

We first need to increase `VERSION` of the serializer to 2 and update `to_dict` accordingly.

In [13]:
@serializer(types=[QuantumElement], public=True)
class QuantumElementSerializer(VersionedClassSerializer[QuantumElement]):
    SERIALIZER_ID = "laboneq.serializers.implementations.QuantumElementSerializer"
    VERSION = 2

    @classmethod
    def to_dict(
        cls, obj: QuantumElement, options: SerializationOptions | None = None
    ) -> JsonSerializableType:
        return {
            "__serializer__": cls.serializer_id(),
            "__version__": cls.version(),
            "__data__": {
                "quantum_element_class": f"{obj.__class__.__module__}.{obj.__class__.__name__}",
                "uid": obj.uid,
                "signals": obj.signals,
                "attribute_class": f"{obj.attributes.__class__.__module__}.{obj.attributes.__class__.__name__}",
                "attribute": attrs.asdict(obj.attributes),
            },
        }

We then add `from_dict_v2` using the new signature of `QuantumElement` class.

In [14]:
@classmethod
def from_dict_v2(
    cls,
    serialized_data: JsonSerializableType,
    options: DeserializationOptions | None = None,
) -> QuantumElement:
    data = serialized_data["__data__"]
    qe_cls = import_cls(data["quantum_element_class"])
    param_cls = import_cls(data["attribute_class"])
    return qe_cls(
        uid=data["uid"],
        signals=data["signals"],
        attribute=param_cls(**from_dict(data["attribute"])),
    )